In [1]:
import pandas as pd

ratings = pd.read_csv("./data/all_ratings_simplified.csv")
print(len(ratings['user'].unique()))

user_counts = ratings.groupby('user').count().reset_index()

filtered_users = user_counts[user_counts['date'] >= 2]['user']
filtered_df = ratings[ratings['user'].isin(filtered_users)]
len(filtered_df['user'].unique())

20537432


9611600

In [2]:
filtered_df.head()

,mark,product,date,user
0,5.0,0001713353,20161003,16201208
1,5.0,0001713353,20160729,225610
2,5.0,0001713353,20160620,18037271
3,5.0,0001713353,20160424,8215727
4,5.0,0001713353,20160214,8084801


In [13]:
import json

unique_products = filtered_df['product'].unique()
product_to_int = {product: i for i, product in enumerate(unique_products)}
filtered_df['product_int'] = filtered_df['product'].map(product_to_int)

int_to_product = {i: product for product, i in product_to_int.items()}
with open("./data/int_to_product.json", "w") as f:
    json.dump(int_to_product, f)

def write_group(group):
    return ' '.join(map(str, group['product_int'].tolist()))

grouped = filtered_df.groupby('user').apply(write_group).reset_index(name='items')

with open('spmf/input.txt', 'w') as f:
    for items in grouped['items']:
        f.write(f"{items}\n")

C:\Users\RINGULARITY\AppData\Local\Temp\ipykernel_8164\930274612.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['product_int'] = filtered_df['product'].map(product_to_int)


In [14]:
import subprocess

spmf_jar_path = r"C:\Users\RINGULARITY\Desktop\epita\eda\eda_project\spmf.jar"

input_file = r"C:\Users\RINGULARITY\Desktop\epita\eda\eda_project\spmf\input.txt"
output_file = r"C:\Users\RINGULARITY\Desktop\epita\eda\eda_project\spmf\apriori.txt"

process = subprocess.Popen([r'C:\Program Files\Java\jdk-20\bin\java', '-jar', spmf_jar_path, 'run', 'Apriori', input_file, output_file, '0.25%'],
                           stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

with open(output_file, 'r') as f:
    for line in f:
        print(line.strip())

print("STDERR:", stderr.decode())

343 #SUP: 4242
358 #SUP: 4242
679 #SUP: 1695
905 #SUP: 1819
1180 #SUP: 1785
1347 #SUP: 4240
1763 #SUP: 2375
1774 #SUP: 1737
2160 #SUP: 2246
2664 #SUP: 1606
2976 #SUP: 4292
3168 #SUP: 2236
3773 #SUP: 5573
3816 #SUP: 1712
4298 #SUP: 2145
4396 #SUP: 4222
4696 #SUP: 4869
4804 #SUP: 2370
4906 #SUP: 1802
5014 #SUP: 2931
6536 #SUP: 1922
6563 #SUP: 6550
6611 #SUP: 2030
6680 #SUP: 2380
6696 #SUP: 2213
6903 #SUP: 2892
7220 #SUP: 1719
7241 #SUP: 2072
7243 #SUP: 2527
7272 #SUP: 1623
7304 #SUP: 2087
7345 #SUP: 2483
7586 #SUP: 6828
7801 #SUP: 1632
7831 #SUP: 2800
8033 #SUP: 7548
8040 #SUP: 7908
10125 #SUP: 4211
10283 #SUP: 1627
10426 #SUP: 1698
10429 #SUP: 1622
10475 #SUP: 2936
10736 #SUP: 1828
11367 #SUP: 2823
11391 #SUP: 2096
11788 #SUP: 2074
12713 #SUP: 2205
12783 #SUP: 1665
12924 #SUP: 2139
12957 #SUP: 4587
13399 #SUP: 2391
13455 #SUP: 1717
13843 #SUP: 4980
13850 #SUP: 3643
14170 #SUP: 1803
14171 #SUP: 1878
15141 #SUP: 1596
15328 #SUP: 2762
15402 #SUP: 1818
15446 #SUP: 2028
15749 #SUP: 2051
1597

In [15]:
items_amount = 2

new_content = ""
with open(output_file, 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        
        if len(line.split(" ")) > items_amount + 1:
            new_line = ""
            for item in line.split("  ")[0].split(" "):
                new_line += int_to_product[int(item)] + " "
            new_line += line.split("  ")[1]
            new_content += new_line

with open("./spmf/filtered_apriori.txt", "w") as f:
    f.write(new_content)

print(new_content)

0003302245 0006155480 #SUP: 4193
0003302245 0007271190 #SUP: 4188
0006155480 0007271190 #SUP: 4213
0060846755 0060846178 #SUP: 7035
0061791644 0300208421 #SUP: 2043
0062571370 0307461068 #SUP: 2000
0062571370 0307594718 #SUP: 2000
0073522465 0312382812 #SUP: 2938
0130231207 0307262812 #SUP: 2582
0307461068 0307594718 #SUP: 3138
0003302245 0006155480 0007271190 #SUP: 4188
0062571370 0307461068 0307594718 #SUP: 2000



In [16]:
import pandas as pd

products = pd.read_csv("./data/all_products_simplified.csv")
products.head()

,id,categories,brand,features,price,date
0,0000092878,[],Keith Graham,[],39.94,19700101
1,000047715X,"[89792, 105868, 122041]",Acp,[],0.00,19700101
2,0000004545,"[89792, 77019, 12065]",Burkhard Jarisch,[],199.99,19700101
3,0000013765,"[89792, 77019, 12065]",Stamps/Baxter,[],0.00,19700101
4,0000477141,"[89792, 25301, 6735]",ACP,[],0.00,19700101


In [17]:
from IPython.display import display

dfs = []

with open("./spmf/filtered_apriori.txt", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        
        result = products[products["id"].isin(line.split(" ")[:-2])].copy()
        result["pattern_amount"] = int(line.split(" ")[-1])
        dfs.append(result)

dfs = sorted(dfs, key=lambda x: x.iloc[0]["pattern_amount"], reverse=True)
for df in dfs:
    display(df)

,id,categories,brand,features,price,date,pattern_amount
22924,0060846178,"[89792, 20014, 117407]",Visit Amazon's Mark Crilley Page,[],8.99,19700101,7035
22944,0060846755,"[89792, 108244, 36162]",Gardner Dozois,[],19.99,19700101,7035


,id,categories,brand,features,price,date,pattern_amount
1428,0006155480,"[89792, 117407, 92546]",Visit Amazon's Dudley Pope Page,[],1.77,19700101,4213
4204,0007271190,[],Visit Amazon's Conn Iggulden Page,[],12.70,19700101,4213


,id,categories,brand,features,price,date,pattern_amount
1005,0003302245,"[89792, 103424, 21337]",Jan Needle,[],0.00,19700101,4193
1428,0006155480,"[89792, 117407, 92546]",Visit Amazon's Dudley Pope Page,[],1.77,19700101,4193


,id,categories,brand,features,price,date,pattern_amount
1005,0003302245,"[89792, 103424, 21337]",Jan Needle,[],0.0,19700101,4188
4204,0007271190,[],Visit Amazon's Conn Iggulden Page,[],12.7,19700101,4188


,id,categories,brand,features,price,date,pattern_amount
1005,0003302245,"[89792, 103424, 21337]",Jan Needle,[],0.00,19700101,4188
1428,0006155480,"[89792, 117407, 92546]",Visit Amazon's Dudley Pope Page,[],1.77,19700101,4188
4204,0007271190,[],Visit Amazon's Conn Iggulden Page,[],12.70,19700101,4188


...

,id,categories,brand,features,price,date,pattern_amount
65935,0130231207,"[89792, 41246, 79314]",Visit Amazon's Robert L. Mott Page,[],0.00,19700101,2582
155783,0307262812,"[89792, 12438, 14269]",Visit Amazon's Wibke Bruhns Page,[],8.92,19700101,2582


,id,categories,brand,features,price,date,pattern_amount
29854,0061791644,"[89792, 6375, 64303]",Visit Amazon's Selena Roberts Page,[],4.74,19700101,2043
151984,0300208421,"[89792, 77019, 9904]",Visit Amazon's Irving Sandler Page,[],49.98,19700101,2043


,id,categories,brand,features,price,date,pattern_amount
37798,0062571370,"[89792, 93327, 56691]",Visit Amazon's Mark Salisbury Page,[],32.02,19700101,2000
158385,0307461068,"[89792, 50012, 59072]",Lily Chin,[],19.95,19700101,2000


,id,categories,brand,features,price,date,pattern_amount
37798,0062571370,"[89792, 93327, 56691]",Visit Amazon's Mark Salisbury Page,[],32.02,19700101,2000
159213,0307594718,"[89792, 108082, 75974]",Visit Amazon's Bruce Hoffman Page,[],11.35,19700101,2000


,id,categories,brand,features,price,date,pattern_amount
37798,0062571370,"[89792, 93327, 56691]",Visit Amazon's Mark Salisbury Page,[],32.02,19700101,2000
158385,0307461068,"[89792, 50012, 59072]",Lily Chin,[],19.95,19700101,2000
159213,0307594718,"[89792, 108082, 75974]",Visit Amazon's Bruce Hoffman Page,[],11.35,19700101,2000


In [18]:
import subprocess

spmf_jar_path = r"C:\Users\RINGULARITY\Desktop\epita\eda\eda_project\spmf.jar"

input_file = r"C:\Users\RINGULARITY\Desktop\epita\eda\eda_project\spmf\input.txt"
output_file = r"C:\Users\RINGULARITY\Desktop\epita\eda\eda_project\spmf\fp_growth.txt"

process = subprocess.Popen([r'C:\Program Files\Java\jdk-20\bin\java', '-jar', spmf_jar_path, 'run', 'FPGrowth_itemsets', input_file, output_file, '1%'],
                           stderr=subprocess.PIPE)

stdout, stderr = process.communicate()
print("STDERR:", stderr.decode())

with open(output_file, 'r') as f:
    for line in f:
        print(line.strip())

STDERR: 
6563 #SUP: 6550
7586 #SUP: 6828
8033 #SUP: 7548
8033 8040 #SUP: 7548
8033 8040 #SUP: 956
8033 8040 8040 #SUP: 956
8033 8040 #SUP: 162
8033 8040 8040 #SUP: 162
8033 8040 8040 #SUP: 162
8033 8040 8040 8040 #SUP: 162
8033 8040 #SUP: 28
8033 8040 8040 #SUP: 28
8033 8040 8040 #SUP: 28
8033 8040 8040 8040 #SUP: 28
8033 8040 8040 #SUP: 28
8033 8040 8040 8040 #SUP: 28
8033 8040 8040 8040 #SUP: 28
8033 8040 8040 8040 8040 #SUP: 28
8040 #SUP: 7908


In [19]:
items_amount = 2

new_content = ""
with open(output_file, 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        
        if len(line.split(" ")) > items_amount + 1:
            new_line = ""
            for item in line.split(" ")[:-2]:
                new_line += int_to_product[int(item)] + " "
            new_line += " " + " ".join(line.split(" ")[-2:])
            new_content += new_line

with open("./spmf/filtered_fp_growth.txt", "w") as f:
    f.write(new_content)

print(new_content)

0060846755 0060846178  #SUP: 7548
0060846755 0060846178  #SUP: 956
0060846755 0060846178 0060846178  #SUP: 956
0060846755 0060846178  #SUP: 162
0060846755 0060846178 0060846178  #SUP: 162
0060846755 0060846178 0060846178  #SUP: 162
0060846755 0060846178 0060846178 0060846178  #SUP: 162
0060846755 0060846178  #SUP: 28
0060846755 0060846178 0060846178  #SUP: 28
0060846755 0060846178 0060846178  #SUP: 28
0060846755 0060846178 0060846178 0060846178  #SUP: 28
0060846755 0060846178 0060846178  #SUP: 28
0060846755 0060846178 0060846178 0060846178  #SUP: 28
0060846755 0060846178 0060846178 0060846178  #SUP: 28
0060846755 0060846178 0060846178 0060846178 0060846178  #SUP: 28



In [20]:
from IPython.display import display

dfs = []

with open("./spmf/filtered_fp_growth.txt", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        
        result = products[products["id"].isin(line.split(" ")[:-2])].copy()
        result["pattern_amount"] = int(line.split(" ")[-1])
        dfs.append(result)

dfs = sorted(dfs, key=lambda x: x.iloc[0]["pattern_amount"], reverse=True)
for df in dfs:
    display(df)

,id,categories,brand,features,price,date,pattern_amount
22924,0060846178,"[89792, 20014, 117407]",Visit Amazon's Mark Crilley Page,[],8.99,19700101,7548
22944,0060846755,"[89792, 108244, 36162]",Gardner Dozois,[],19.99,19700101,7548


,id,categories,brand,features,price,date,pattern_amount
22924,0060846178,"[89792, 20014, 117407]",Visit Amazon's Mark Crilley Page,[],8.99,19700101,956
22944,0060846755,"[89792, 108244, 36162]",Gardner Dozois,[],19.99,19700101,956


,id,categories,brand,features,price,date,pattern_amount
22924,0060846178,"[89792, 20014, 117407]",Visit Amazon's Mark Crilley Page,[],8.99,19700101,956
22944,0060846755,"[89792, 108244, 36162]",Gardner Dozois,[],19.99,19700101,956


,id,categories,brand,features,price,date,pattern_amount
22924,0060846178,"[89792, 20014, 117407]",Visit Amazon's Mark Crilley Page,[],8.99,19700101,162
22944,0060846755,"[89792, 108244, 36162]",Gardner Dozois,[],19.99,19700101,162


,id,categories,brand,features,price,date,pattern_amount
22924,0060846178,"[89792, 20014, 117407]",Visit Amazon's Mark Crilley Page,[],8.99,19700101,162
22944,0060846755,"[89792, 108244, 36162]",Gardner Dozois,[],19.99,19700101,162


...

,id,categories,brand,features,price,date,pattern_amount
22924,0060846178,"[89792, 20014, 117407]",Visit Amazon's Mark Crilley Page,[],8.99,19700101,28
22944,0060846755,"[89792, 108244, 36162]",Gardner Dozois,[],19.99,19700101,28


,id,categories,brand,features,price,date,pattern_amount
22924,0060846178,"[89792, 20014, 117407]",Visit Amazon's Mark Crilley Page,[],8.99,19700101,28
22944,0060846755,"[89792, 108244, 36162]",Gardner Dozois,[],19.99,19700101,28


,id,categories,brand,features,price,date,pattern_amount
22924,0060846178,"[89792, 20014, 117407]",Visit Amazon's Mark Crilley Page,[],8.99,19700101,28
22944,0060846755,"[89792, 108244, 36162]",Gardner Dozois,[],19.99,19700101,28


,id,categories,brand,features,price,date,pattern_amount
22924,0060846178,"[89792, 20014, 117407]",Visit Amazon's Mark Crilley Page,[],8.99,19700101,28
22944,0060846755,"[89792, 108244, 36162]",Gardner Dozois,[],19.99,19700101,28


,id,categories,brand,features,price,date,pattern_amount
22924,0060846178,"[89792, 20014, 117407]",Visit Amazon's Mark Crilley Page,[],8.99,19700101,28
22944,0060846755,"[89792, 108244, 36162]",Gardner Dozois,[],19.99,19700101,28


In [21]:
import subprocess

spmf_jar_path = r"C:\Users\RINGULARITY\Desktop\epita\eda\eda_project\spmf\spmf.jar"

input_file = r"C:\Users\RINGULARITY\Desktop\epita\eda\eda_project\spmf\input.txt"
output_file = r"C:\Users\RINGULARITY\Desktop\epita\eda\eda_project\spmf\n_eclat_closed.txt"

process = subprocess.Popen([r'C:\Program Files\Java\jdk-20\bin\java', '-jar', spmf_jar_path, 'run', 'NEclatClosed', input_file, output_file, '0.25%'],
                           stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
print("STDERR:", stderr.decode())

with open(output_file, 'r') as f:
    for line in f:
        print(line.strip())

items_amount = 2

new_content = ""
with open(output_file, 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        
        if len(line.split(" ")) > items_amount + 1:
            new_line = ""
            for item in line.split(" ")[:-2]:
                new_line += int_to_product[int(item)] + " "
            new_line += " " + " ".join(line.split(" ")[-2:])
            new_content += new_line

with open("./spmf/filtered_n_eclat_closed.txt", "w") as f:
    f.write(new_content)


dfs = []

with open("./spmf/filtered_n_eclat_closed.txt", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        
        result = products[products["id"].isin(line.split(" ")[:-2])].copy()
        result["pattern_amount"] = int(line.split(" ")[-1])
        dfs.append(result)

dfs = sorted(dfs, key=lambda x: x.iloc[0]["pattern_amount"], reverse=True)
for df in dfs:
    display(df)

STDERR: 
29243 #SUP: 1593
15141 #SUP: 1569
24367 #SUP: 1589
2664 #SUP: 1606
25442 #SUP: 1603
34691 #SUP: 1614
17225 #SUP: 1615
42999 #SUP: 1463
10429 #SUP: 1622
7272 #SUP: 802
21916 #SUP: 1625
10283 #SUP: 1621
7801 #SUP: 818
38507 #SUP: 1637
22316 #SUP: 1649
36595 #SUP: 1662
12783 #SUP: 1663
25306 #SUP: 1666
49335 #SUP: 1663
33776 #SUP: 1671
23933 #SUP: 1681
26027 #SUP: 1689
30312 #SUP: 1690
679 #SUP: 1695
27101 #SUP: 1691
10426 #SUP: 1614
3816 #SUP: 1711
13455 #SUP: 1704
7220 #SUP: 847
25127 #SUP: 1728
45718 #SUP: 1729
1774 #SUP: 1730
20270 #SUP: 1738
21514 #SUP: 1683
19940 #SUP: 1745
37670 #SUP: 1749
16075 #SUP: 1761
1180 #SUP: 1784
37228 #SUP: 1778
21592 #SUP: 1801
4906 #SUP: 1802
14170 #SUP: 1802
15402 #SUP: 1818
905 #SUP: 1768
10736 #SUP: 1816
46073 #SUP: 1847
16205 #SUP: 1849
22453 #SUP: 1859
14171 #SUP: 1878
29959 #SUP: 1883
45750 #SUP: 1905
6536 #SUP: 962
18178 #SUP: 1947
45336 #SUP: 1952
34762 #SUP: 1970
17138 #SUP: 1992
48534 #SUP: 1993
22216 #SUP: 2006
15446 #SUP: 2027
6611 

,id,categories,brand,features,price,date,pattern_amount
22924,0060846178,"[89792, 20014, 117407]",Visit Amazon's Mark Crilley Page,[],8.99,19700101,7047
22944,0060846755,"[89792, 108244, 36162]",Gardner Dozois,[],19.99,19700101,7047


,id,categories,brand,features,price,date,pattern_amount
1005,0003302245,"[89792, 103424, 21337]",Jan Needle,[],0.00,19700101,4228
1428,0006155480,"[89792, 117407, 92546]",Visit Amazon's Dudley Pope Page,[],1.77,19700101,4228


,id,categories,brand,features,price,date,pattern_amount
1005,0003302245,"[89792, 103424, 21337]",Jan Needle,[],0.00,19700101,4226
1428,0006155480,"[89792, 117407, 92546]",Visit Amazon's Dudley Pope Page,[],1.77,19700101,4226
4204,0007271190,[],Visit Amazon's Conn Iggulden Page,[],12.70,19700101,4226


,id,categories,brand,features,price,date,pattern_amount
158385,0307461068,"[89792, 50012, 59072]",Lily Chin,[],19.95,19700101,3141
159213,0307594718,"[89792, 108082, 75974]",Visit Amazon's Bruce Hoffman Page,[],11.35,19700101,3141


,id,categories,brand,features,price,date,pattern_amount
54357,0073522465,"[89792, 12816, 35623]",Robert Feldman,[],0.00,19700101,2944
176458,0312382812,"[89792, 31458, 45118]",Visit Amazon's Sibley Miller Page,[],7.99,19700101,2944


,id,categories,brand,features,price,date,pattern_amount
65935,0130231207,"[89792, 41246, 79314]",Visit Amazon's Robert L. Mott Page,[],0.00,19700101,2590
155783,0307262812,"[89792, 12438, 14269]",Visit Amazon's Wibke Bruhns Page,[],8.92,19700101,2590


,id,categories,brand,features,price,date,pattern_amount
29854,0061791644,"[89792, 6375, 64303]",Visit Amazon's Selena Roberts Page,[],4.74,19700101,2047
151984,0300208421,"[89792, 77019, 9904]",Visit Amazon's Irving Sandler Page,[],49.98,19700101,2047


,id,categories,brand,features,price,date,pattern_amount
37798,0062571370,"[89792, 93327, 56691]",Visit Amazon's Mark Salisbury Page,[],32.02,19700101,2003
158385,0307461068,"[89792, 50012, 59072]",Lily Chin,[],19.95,19700101,2003
159213,0307594718,"[89792, 108082, 75974]",Visit Amazon's Bruce Hoffman Page,[],11.35,19700101,2003


In [22]:
import subprocess

spmf_jar_path = r"C:\Users\RINGULARITY\Desktop\epita\eda\eda_project\spmf.jar"

input_file = r"C:\Users\RINGULARITY\Desktop\epita\eda\eda_project\spmf\input.txt"
output_file = r"C:\Users\RINGULARITY\Desktop\epita\eda\eda_project\spmf\neg_fin.txt"

process = subprocess.Popen([r'C:\Program Files\Java\jdk-20\bin\java', '-jar', spmf_jar_path, 'run', 'NegFIN', input_file, output_file, '0.1%', '0.00001%'],
                           stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
print("STDERR:", stderr.decode())

with open(output_file, 'r') as f:
    for line in f:
        print(line.strip())

items_amount = 2

new_content = ""
with open(output_file, 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        
        if len(line.split(" ")) > items_amount + 1:
            new_line = ""
            for item in line.split(" ")[:-2]:
                new_line += int_to_product[int(item)] + " "
            new_line += " " + " ".join(line.split(" ")[-2:])
            new_content += new_line

with open("./spmf/filtered_neg_fin.txt", "w") as f:
    f.write(new_content)


dfs = []

with open("./spmf/filtered_neg_fin.txt", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        
        result = products[products["id"].isin(line.split(" ")[:-2])].copy()
        result["pattern_amount"] = int(line.split(" ")[-1])
        dfs.append(result)

dfs = sorted(dfs, key=lambda x: x.iloc[0]["pattern_amount"], reverse=True)
for df in dfs:
    display(df)

STDERR: 
49819 #SUP: 639
11637 #SUP: 639
10780 #SUP: 639
30924 #SUP: 640
15611 #SUP: 640
50452 #SUP: 642
13355 #SUP: 643
11804 #SUP: 644
11804 45669 #SUP: 643
32842 #SUP: 645
10093 #SUP: 645
5899 #SUP: 645
45669 #SUP: 646
12075 #SUP: 646
17985 #SUP: 648
7723 #SUP: 648
41728 #SUP: 649
40180 #SUP: 649
28646 #SUP: 650
8839 #SUP: 650
42250 #SUP: 652
7452 #SUP: 652
31280 #SUP: 653
45573 #SUP: 654
34096 #SUP: 654
45311 #SUP: 655
41039 #SUP: 655
16365 #SUP: 655
46086 #SUP: 656
22529 #SUP: 656
16035 #SUP: 656
7111 #SUP: 657
13253 #SUP: 658
13000 #SUP: 659
41062 #SUP: 660
15592 #SUP: 660
34239 #SUP: 661
14904 #SUP: 662
7702 #SUP: 662
6894 #SUP: 662
18524 #SUP: 664
35632 #SUP: 665
28326 #SUP: 666
651 #SUP: 666
54415 #SUP: 667
47289 #SUP: 668
23394 #SUP: 668
6492 #SUP: 669
42449 #SUP: 671
11035 #SUP: 671
43221 #SUP: 672
37866 #SUP: 672
6374 #SUP: 672
8830 #SUP: 673
15373 #SUP: 674
379 #SUP: 674
13136 #SUP: 675
35376 #SUP: 676
9572 #SUP: 676
1624 #SUP: 677
25474 #SUP: 678
45823 #SUP: 679
27510 #SU

,id,categories,brand,features,price,date,pattern_amount
22924,0060846178,"[89792, 20014, 117407]",Visit Amazon's Mark Crilley Page,[],8.99,19700101,7548
22944,0060846755,"[89792, 108244, 36162]",Gardner Dozois,[],19.99,19700101,7548


,id,categories,brand,features,price,date,pattern_amount
1005,0003302245,"[89792, 103424, 21337]",Jan Needle,[],0.00,19700101,4242
1428,0006155480,"[89792, 117407, 92546]",Visit Amazon's Dudley Pope Page,[],1.77,19700101,4242


,id,categories,brand,features,price,date,pattern_amount
1428,0006155480,"[89792, 117407, 92546]",Visit Amazon's Dudley Pope Page,[],1.77,19700101,4240
4204,0007271190,[],Visit Amazon's Conn Iggulden Page,[],12.70,19700101,4240


,id,categories,brand,features,price,date,pattern_amount
1005,0003302245,"[89792, 103424, 21337]",Jan Needle,[],0.0,19700101,4240
4204,0007271190,[],Visit Amazon's Conn Iggulden Page,[],12.7,19700101,4240


...

,id,categories,brand,features,price,date,pattern_amount
43055,0071106359,"[89792, 13092, 69095]",Visit Amazon's David M. Burton Page,[],0.00,19700101,702
174650,0312338961,"[89792, 118319, 83967]",Visit Amazon's Randy Sutton Page,[],5.99,19700101,702


,id,categories,brand,features,price,date,pattern_amount
46824,0071494138,"[89792, 108082, 75974]",Visit Amazon's Aamir A. Rehman Page,[],0.00,19700101,695
155893,0307265242,"[89792, 118319, 83967]",Visit Amazon's Steven E. Woodworth Page,[],37.88,19700101,695


,id,categories,brand,features,price,date,pattern_amount
69560,0131408836,"[89792, 105868, 16597]",Visit Amazon's Deborah Phillips Page,[],0.00,19700101,694
160175,0307890074,"[89792, 64446, 4356]",Visit Amazon's Prima Games Page,[],36.98,19700101,694


,id,categories,brand,features,price,date,pattern_amount
6564,0008237972,"[89792, 117407, 92546]",Visit Amazon's Zara Stoneley Page,[],11.99,19700101,691
100282,0174434804,"[89792, 117407, 74808]",William Shakespeare,[],29.95,19700101,691


,id,categories,brand,features,price,date,pattern_amount
30071,0061826871,"[89792, 108244, 102065]",Visit Amazon's Melissa Marr Page,[],0.00,19700101,643
155868,030726291X,"[89792, 117407, 92546]",Visit Amazon's P. D. James Page,[],4.99,19700101,643
